In [1]:
import cv2
import numpy as np
import sys


## Utilities

In [2]:
def frame_differencing(prev_frame, curr_frame):
    """Calculate frame difference."""
    diff = cv2.absdiff(curr_frame, prev_frame)
    gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 30, 255, cv2.THRESH_BINARY)
    return thresh

def skin_detection(frame):
    """Detect skin in the given frame."""
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    lower_skin = np.array([0, 40, 60], dtype=np.uint8)
    upper_skin = np.array([20, 255, 255], dtype=np.uint8)
    mask = cv2.inRange(hsv, lower_skin, upper_skin)
    return mask

def motion_energy(frames):
    """Calculate motion energy across a list of frames."""
    motion_history = np.zeros(frames[0].shape[:2], dtype=np.float32)
    for frame in frames:
        motion_history = cv2.motempl.updateMotionHistory(frame, motion_history, 1, 5)
    return motion_history

def accumulate_motion_energy(frames):
    """Accumulate motion energy from a list of binary difference frames."""
    # Assuming frames are binary (0 or 255), where 255 indicates motion
    # Initialize an accumulator frame with zeros
    acc = np.zeros_like(frames[0], dtype=np.float32)
    
    # Accumulate frames by adding
    for frame in frames:
        # Convert frame to float to prevent data type overflow
        acc = acc + frame.astype(np.float32)
    
    # Normalize the accumulated image to the range [0, 255]
    acc = np.clip((acc / len(frames)), 0, 255).astype(np.uint8)
    
    return acc


## Main

In [3]:

def main():
    cap = cv2.VideoCapture(0)  # Use 0 for webcam

    if not cap.isOpened():
        print("Cannot open camera")
        sys.exit()

    ret, prev_frame = cap.read()
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        return

    prev_frame = cv2.flip(prev_frame, 1)  # Flip frame
    frames_for_motion = []

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.flip(frame, 1)  # Flip frame for a mirror effect
        frame_diff = frame_differencing(prev_frame, frame)
        skin_mask = skin_detection(frame)
        cv2.imshow('Skin Mask', skin_mask)
        # Combine frame difference and skin detection for better results
        combined = cv2.bitwise_and(frame_diff, frame_diff, mask=skin_mask)

        cv2.imshow('Frame Difference', combined)

        frames_for_motion.append(combined)
        if len(frames_for_motion) > 5:  # Keeping the last 5 frames
            frames_for_motion.pop(0)

        # Calculate and visualize accumulated motion energy (optional)
        if len(frames_for_motion) >= 5:  # Ensure there are enough frames
            accumulated_me = accumulate_motion_energy(frames_for_motion)
            cv2.imshow('Accumulated Motion Energy', accumulated_me)


        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        prev_frame = frame

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()
